# Ejercicios

In [ ]:
library(rentrez)
library(httr)
library(ggplot2)
library(xml2)

## Ejercicio 1

Utilice el servicio EInfo de Entrez para construir una tabla que contenga el nombre de la base de datos,
la descripción, el número de registros, y la fecha de la última modificación para todas las bases de datos
disponibles.

In [ ]:
dbs <- entrez_dbs()

In [ ]:
db_data <- list()
for (db in dbs[1:3]) {
    data <- entrez_db_summary(db)
    elements <- c(data["DbName"], data["Description"], data["Count"], data["LastUpdate"])
    db_data[[length(db_data)+1]] <- elements
}

In [ ]:
do.call(rbind, db_data)
# t(sapply(db_data, function(x) x))

## Ejercicio 2

Obtenga de la base de datos 'nuccore' de NCBI el número de registros que corresponden con secuencias
del virus HIV-1 que fueron publicadas en cada año desde 1980 hasta 2020. Haga un gráfico de barras para
mostrar estos resultados.

Ayuda: ?barplot

In [ ]:
years <- 1980:2020
counter <- c()
for (year in years) {
    Sys.sleep(0.3)
    query <- paste("Human immunodeficiency virus 1[organism] AND", year)
    respuesta <- entrez_search(
        db='nuccore',
        term=query
    )
    counter <- c(counter, respuesta$count)
}

In [ ]:
ggplot(data=data.frame(counter), aes(x=years, y=counter)) + geom_bar(stat="identity")

## Ejercicio 3

Buscar información sobre la base de datos MESH (NCBI). ¿Qué información almacena?

- Buscar en esta base de datos el nombre del gen relacionado con la enfermedad Fibrosis Quística (Cystic fibrosis).
  - Ayuda: Buscar campo de búsqueda 'MeSH Terms'.
  - Ayuda: Usar rettype='txt'
  - Ayuda: El nombre del gen aparece en el texto de la descripción.
- Buscar en la base de datos 'gene' de NCBI, los registros de este gen en el ser humano (en formato XML).
- Recuperar la lista de exones. ¿En qué locus está ubicado el gen?.
  - Ayuda: Buscar rettype=gene_table
- Buscar en Swiss-Prot los registros que están asociados a esta proteína.
  - Ayuda: Mapear los ID de genes de NCBI/Entrez a Uniprot.
  - Ayuda: Buscar los tipos de Identificadores correspondientes de la web.
- Recuperar la secuencias de la proteína

In [ ]:
query <- "Cystic fibrosis[MeSH Terms]"
respuesta <- entrez_search(
    db='mesh',
    term=query
)

In [ ]:
respuesta2 <- entrez_fetch(db="mesh", id=respuesta$ids, rettype="txt")

In [ ]:
cat(respuesta2)

In [ ]:
query <- "CYSTIC FIBROSIS TRANSMEMBRANE CONDUCTANCE REGULATOR[Title] AND human[orgn]"
respuesta <- entrez_search(
    db='gene',
    term=query
)


In [ ]:
respuesta2 <- entrez_fetch(db="gene", id=respuesta$ids[1], rettype="gene_table")

In [ ]:
cat(respuesta2)

In [ ]:
retrieve_url <- 'https://www.uniprot.org/uploadlists'
respuesta3 <- GET(
    url=retrieve_url,
    query=list(
        query=respuesta$ids[1],
        format="tab",
        from='P_ENTREZGENEID',
        to='ID'
    )
)

In [ ]:
respuesta3 <- read.table(text=content(respuesta3), header=TRUE)

In [ ]:
respuesta3$To

In [ ]:
retrieve_url <- 'https://www.uniprot.org/uploadlists'
respuesta4 <- GET(
    url=retrieve_url,
    query=list(
        query=paste(respuesta3$To, collapse=" "),
        format="fasta",
        from='ID',
        to='ACC'
    )
)

In [ ]:
cat(content(respuesta4))

## Ejercicio 4

- Recuperar los registros que corresponden a secuencias nucleotídicas de genomas completos de virus de la familia Flaviviridae. 
- Trabajar con 100.
- Generar un histograma de los tamaños de las secuencias.
- Obtener de cada registro el código del 'TaxID'. Buscar en la base de datos 'Taxonomy' el nombre científico, el rank y la division de cada uno.

- ¿Cuantos corresponde a aislamientos?


In [ ]:
respuesta <- entrez_search(
    db='nuccore',
    term="flaviviridae[orgn] AND complete genome[Title]",
    retmax=10000
)

In [ ]:
respuesta2 <- entrez_summary(
    db='nuccore',
    id=respuesta$ids[1:200],
    rettype='xmls'
)

In [ ]:
flavi <- sapply(
    respuesta2,
    function(x) unlist(x[c('slen', 'taxid', 'organism')])
)
flavi <- as.data.frame(t(flavi))
head(flavi)

In [ ]:
hist(as.numeric(flavi$slen), 20)

In [ ]:
flavi_tax <- entrez_summary(
    db='taxonomy',
    id=flavi$taxid
)

In [ ]:
flavi2 <- sapply(
    flavi_tax,
    function(x) unlist(x[c('rank', 'division', 'scientificname')])
)
flavi2 <- as.data.frame(t(flavi2))

In [ ]:
head(flavi2)

## Ejercicio 5

Recuperar todos los registros que corresponden a secuencias de 'Amapari virus' en la base de datos
'nuc_core'. Recuperar de ellos los identificadores de las publicaciones relacionadas. Obtener de 'pubmed'
el título del artículo, los autores.


In [ ]:
respuesta <- entrez_search(
    db='nuccore',
    term='Amapari virus[orgn]',
    retmax=30
)

In [ ]:
respuesta$ids

In [ ]:
resumenes <- entrez_fetch(
    db='nuccore',
    id=respuesta$ids,
    rettype="text",
    retmode="xml"
)

In [ ]:
pubmed_links <- entrez_link(
    id=respuesta$ids,
    dbfrom='nuccore',
    db="pubmed"
)


In [ ]:

pubmed_links$links$nuccore_pubmed

resumenes <- entrez_summary(
    db="pubmed",
    id=pubmed_links$links$nuccore_pubmed
)

In [ ]:
pubmed_data <- sapply(
    resumenes,
    function(x) c(x[[c('title')]], paste(x[[c('authors')]]$name, collapse=", "))
)
pubmed_data <- t(pubmed_data)

In [ ]:
pubmed_data

## Ejercicio 6

Buscar en base de datos 'SRA' (Sequence Read Archive) de NCBI los registros que corresponden a
los proyectos de secuenciación de virus de la familia Baculoviridae (Trabajar con un máximo de 30).
Recuperar de cada ellos el nómbre del organismo secuenciado y su 'TaxId' y el título del proyecto.

In [ ]:
respuesta <- entrez_search(
    db='sra',
    term='baculoviridae[orgn]',
    retmax=30
)

In [ ]:
resumenes <- entrez_fetch(
    db='sra',
    id=respuesta$ids,
    rettype="xml"
)

In [ ]:
sra_data <- as_list(read_xml(resumenes))

In [ ]:
head(t(sapply(
    sra_data$EXPERIMENT_PACKAGE_SET,
    function(x) c(
        x$EXPERIMENT$'TITLE',
        x$SAMPLE$'SAMPLE_NAME'$'TAXON_ID',
        x$SAMPLE$'SAMPLE_NAME'$SCIENTIFIC_NAME
    )
)))